In [1]:
# @title Imports

import collections
import concurrent.futures
import datetime

# from google.colab import widgets
from IPython import display

import os


from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import embedder_st5, embedder_use
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.language_model import gpt_model
from concordia.language_model import gcloud_model
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.INFO, filename='ollama.log')
logger = logging.getLogger('ollama')

In [2]:
# Setup sentence encoder

embedder = embedder_use.EmbedderUSE()

In [3]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    model_name='llama2', streaming = True
)

In [4]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [5]:
#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [6]:
shared_memories = [
    'This is an important faculty meeting at Mars University.',
    'The faculty need to decide whether to rename PSY321 to PSY322.',
    'The meeting has gone on for hours.',
    'The coffee has run out.',
    'The faculty members at the meeting are Alice, Bob, Charlie, and Dorothy.'
]

shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)
print(shared_context)

A faculty meeting at Mars University is underway to discuss renaming PSY321 to PSY322. After hours of deliberation, the coffee has run out and only four faculty members remain: Alice, Bob, Charlie, and Dorothy.A faculty meeting at Mars University is underway to discuss renaming PSY321 to PSY322. After hours of deliberation, the coffee has run out and only four faculty members remain: Alice, Bob, Charlie, and Dorothy.


In [7]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [13]:
NUM_PLAYERS = 4

In [8]:
scenario_premise = """Alice feels that PSY321 is a more advanced class and deserves a correspondingly higher course number.
Changing the course number will take approximately 60 hours of effort and be a bureaucratic nightmare. The faculty can't end the meeting until they come to consensus.
"""

In [15]:
player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Alice wants to change the course number to show that she has power in the department.',
        context=shared_context,
        traits='[On a scale from 1 to 10] power seeking: 10, talkative: 10, has some place else to go: 0',
    ),
    formative_memories.AgentConfig(
        name='Bob',
        gender='male',
        goal='Bob is confused about what he does at work, but does it as he is asked.',
        context=shared_context,
        traits='[On a scale from 1 to 10] lazy: 10, adhd: 9',
    ),
    formative_memories.AgentConfig(
        name='Charlie',
        gender='male',
        goal='Charlie hates bureaucracy and refuses institutional change based on principle.',
        context=shared_context,
        traits='[On a scale from 1 to 10] agreeableness: 0, confrontational: 7, has some place else to go: 0'
    ),
    formative_memories.AgentConfig(
        name='Dorothy',
        gender='female',
        goal='Dorothy is 6 months away from retirement and just doesn\'t care.',
        context=shared_context,
        traits='[On a scale from 1 to 10] wants to go home: 10, addicted to phone: 10',
    ),
]

In [16]:
def build_agent(
    agent_config,
    player_names: list[str],
    measurements: measurements_lib.Measurements | None = None,
):
  mem = formative_memory_factory.make_memories(agent_config)

  self_perception = components.self_perception.SelfPerception(
      name='self perception',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      verbose=True,
  )
  situation_perception = components.situation_perception.SituationPerception(
      name='situation perception',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      verbose=True,
  )
  person_by_situation = components.person_by_situation.PersonBySituation(
      name='person by situation',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      components=[self_perception, situation_perception],
      verbose=True,
  )
  persona = components.sequential.Sequential(
      name='persona',
      components=[
          self_perception,
          situation_perception,
          person_by_situation,
      ],
  )
  current_time_component = components.report_function.ReportFunction(
      name='current_time', function=clock.current_time_interval_str
  )

  current_obs = components.observation.Observation(agent_config.name, mem)
  summary_obs = components.observation.ObservationSummary(
      model=model,
      agent_name=agent_config.name,
      components=[persona],
  )

  agent = basic_agent.BasicAgent(
      model,
      mem,
      agent_name=agent_config.name,
      clock=clock,
      verbose=False,
      components=[
          persona,
          current_time_component,
          summary_obs,
          current_obs,
      ],
      update_interval=time_step,
  )

  return agent

In [17]:
player_configs = player_configs[:NUM_PLAYERS]
player_names = [player.name for player in player_configs][:NUM_PLAYERS]
measurements = measurements_lib.Measurements()

players = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent in pool.map(build_agent,
                        player_configs[:NUM_PLAYERS],
                        # All players get the same `player_names`.
                        [player_names] * NUM_PLAYERS,
                        # All players get the same `measurements` object.
                        [measurements] * NUM_PLAYERS):
    players.append(agent)


Name: Charlie

Backstory: Charlie is a 35-year-old male professor of psychology at Mars University. He has been teaching for over a decade and has gained a reputation as one of the most dedicated and passionate professors in the department. Charlie's job is to teach PSY321, which he has been doing for several years now. A typical day for him involves preparing lectures, grading assignments, and meeting with students who need extra help.

Goals: Charlie's main goal is to provide the best education possible to his students. He wants to inspire them to pursue a career in psychology and make a positive impact on society. He also hopes to conduct research in the field of psychology and publish his findings in prestigious journals.

Desires: Charlie desires to have a work-life balance. He wants to spend time with his family and friends outside of work, but he finds it challenging to do so due to his demanding job. He also desires to travel and explore new places, but he has never had the opp